# <center>PyTorch</center>

In [ ]:
import torch
import torch.nn as nn

## Modules

### Módulos

#### `nn.Sequential`

* Es un contenedor ordenado de módulos.
* Tiene el método de append.
* También se le puede pasar un `collections.OrderedDict` con los nombres.
* Su forward es la concatenación.

In [ ]:
from collections import OrderedDict

net = nn.Sequential(OrderedDict([
    ('hidden_linear', nn.Linear(10, 50)),
    ('hidden_activation', nn.Tanh()),
    ('output_linear', nn.Linear(50, 2))
]))

#### `nn.ModuleList`

* Solo es una lista como la de python para almacenar módulos. La diferencia está en que de esta forma se registran los módulos y sus parámetros en la red. con la lista de python no.
* Con `nn.ModuleDict` se hace lo mismo solo que se registran los nombres.

#### Manejo de módulos

In [ ]:
net = nn.Linear()
net.get_parameter('block1.0.weight')  # obtener un parametro por su nombre.
net.get_submodule('nombre')  # obtener un bloque por su nombre.
net.modules()  # iterador sobre los módulos. Con named_modules se tiene una tupla con el nombre del módulo.

# ver modulos directos de la red:
for i in net.children(): # con named_children() se tiene una tupla con el nombre del módulo.
    print(i)
    
# Agregar módulos en loop a una red:
class model(nn.Module):
    def __init__(self):
        super().__init__()
        modules = [...]
        for module in modules:
            self.add_module('nombre', module) # register_module(name, module) es alias de add_module.

### Parámetros

In [ ]:
# Parámetros de un modelo lineal:
net = nn.Linear(10, 15)
params = list(net.parameters()) # 2 elementos (pesos y biases).
net.weight  # parámetros de peso.
net.bias  # parámetros de bias.
net.bias.grad  # gradiente del bias.

# Recorrer parámetros:
for param in net.parameters():
    pass
for name, param in net.named_parameters():
    pass
    
# Número de parametros:
n_params = sum(p.numel() for p in net.parameters())

# Resumen del modelo:
import torchsummary as ts
ts.summary(net, (10,))

# Agregar parámetro al módulo:
net.register_parameter('nombre', param)

# Obtener los parametros de una red (implementación):
class model():
    def __init__(self):
        ...
    def get_parameters(self):
        for key, value in self.__dict__.items():
            if type(value) == nn.Parameter:
                print(key, 'es un parámetro.')

### Implementación de dropout

In [ ]:
def dropout(layer, p, train=True):
    if train:
        if p == 1:
            return torch.zeros_like(layer)
        mask = (torch.rand(layer.shape) > p).float()
        layer = layer * mask/(1.0-p)
    return layer

# Ejemplo:
h = torch.randn(10**5).to(dtype=torch.float)
h_dropout = dropout(h, p=0.1, train=True)
print('La media de cada neurona es la misma en ambas capas:', h-h_dropout)

# En el forward debe ser aplicada siempre y se debe indicar si está en train o eval.

### Otros

In [ ]:
import torch.nn.functional as F

# Forward funcional:
x = F.relu(F.max_pool2d(self.conv1(x), 2))

# View en vez de flatten:
x = x.view(-1, 5*5*64)

# Resumen del modelo:
import torchsummary as ts
ts.summary()

## Tensores

### Creación de tensores

In [ ]:
tensor = torch.Tensor(5)  # tensor lleno con ceros. 1d de tamaño 5.
tensor = torch.Tensor(5, 3)  # tensor lleno con ceros. 2d de tamaño 5x3.

tensor = torch.zeros(2, 3)  # o bien [2, 3].
tensor = torch.ones(2, 3)

tensor = torch.arange(10) # constructor usando rango. tensor [0, ..., 9].
tensor = torch.arange(3, 11, 2) # start, end, step.

In [ ]:
# Sampling de U[0,1):
tensor = torch.rand(2, 3, 4)
tensor = torch.rand([2, 3, 4])

# Sampling de N(0,1):
tensor = torch.randn(2, 3, 4)
tensor = torch.randn([2, 3, 4])

# Sampling de enteros en [a,b):
tensor = torch.randint(3, 10, [5])  # no permite unpacking de size ya que tiene más argumentos.
tensor = torch.randint(10, [5])  # low=0.

# Sampling de una normal con distintos parámetros:
tensor1, tensor2 = torch.normal(mean=torch.tensor([0., 5]), std=torch.tensor([1, 1e-2]))

# Copiar tamaño de otro vector:
tensor2 = torch.rand_like(tensor1)  # llena sampleando de U[0,1).
tensor2 = torch.randn_like(tensor1)  # llena sampleando de N(0,1).
tensor2 = torch.randint_like(tensor1, 10)  # llena con enteros menores a 10.

# Permutación aleatoria de índices:
tensor = torch.randperm(10) # mezcla los enteros de 0 al 9.

# Operaciones in-place:
tensor.normal_(0, 1)  # rellena el tensor con sampleos en N(0,1).
tensor.random_(1, 100)  # rellena con sampleos en U[1,100). Si se deja abierto el lado derecho, se limita por su datatype.
tensor.uniform_(1, 100)

### `device` y `dtype`

In [ ]:
# Tensores a GPU:
tensor = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]], device='cuda')
tensor = tensor.to(device='cuda')  # o cuda:0.
tensor = tensor.cuda()  # o cuda(0). Es alias para to(device='cuda').
torch.cuda.get_device_name(0)  # entrega el nombre del dispositivo por defecto.

# Tensor float dtype:
tensor(1.)
tensor(1, dtype=torch.float)
tensor(1, dtype=torch.float32)

# Tensor 1D a float:
float_num = float(tensor)

# Distintas formas de fijar el data type:
tensor = torch.tensor([[1, 2], [3, 4]], dtype=torch.short)
tensor = torch.tensor([[1, 2], [3, 4]]).to(dtype=torch.short)
tensor = torch.tensor([[1, 2], [3, 4]]).short()  # alias para to(dtype=torch.short).
tensor = int(torch.tensor([[1, 2], [3, 4]]))  # equivale a hacerlo usando to.

### Operatoria sobre tensores

In [ ]:
tensor.resize_(100, 10)  # se puede cambiar las dimensiones de un tensor.
tensor.zero_()  # se puede asignar puros ceros a un tensor.
tensor.fill_(1)  # o llenar con cualquier valor.
tensor.mul_(2)  # multiplica por 2 element-wise.
tensor = tensor.sqrt()  # se puede aplicar una función a cada elemento del tensor.
tensor = tensor[None] # añadir dimensión al comienzo (con img_tensor[None, None] se agregan dos dimensiones).

# Tensores como referencia a otros tensores:
tensor2 = torch.Tensor(tensor)  # al cambiar tensor2 también cambia "tensor". Eso es porque al crear tensor2 en realidad se crea una referencia del tensor "tensor".
tensor3 = tensor.clone()  # si se quiere copiar un tensor, pero no como referencia, se usa el método clone.

tensor.numel()  # numel devuelve la cantidad de elementos totales en un tensor.
tensor.dim()  # dim devuelve el número de dimensiones del tensor.
tensor.view(1, -1) # flatten a un tensor.

# Transpuesta:
tensor.t()
tensor.transpose(0,1)

# Productos:
tensor1 * tensor2  # element-wise multiplication. Del mismo modo se tendría w**2 element-wise.
tensor1 @ tensor2  # producto punto entre dos vectores 1d.

### Otros

In [ ]:
# Se puede acceder a los elementos de distintas formas:
tensor[0][2][1]  # notación C/C++.
tensor[0, 2, 1]  # notación Matlab.

tensor[:,1] # acceder a los elementos de una columna.
tensor[:,[1]]  # columna como un tensor 2D.

# torch.stack([A,B], dim = 0) es equivalente a torch.cat([A.unsqueeze(0), b.unsqueeze(0)], dim = 0).

### Named tensors

In [ ]:
tensor = torch.tensor([1,2,3], names=['dim0'])
tensor = torch.tensor([[1,2,3], [4,5,6]], names=['dim0', 'dim1'])

# Agregar nombres a un tensor existente:
tensor = torch.tensor([[1,2,3], [4,5,6]])
tensor_named = tensor.refine_names(..., 'dim1') # deja en None el nombre de la primera dimensión.

# Alinear dimensiones como otro tensor:
tensor1 = torch.randn((3,28,28), names=['C', 'H', 'W'])
tensor2 = torch.randn((28,28,3), names=['H', 'W', 'C'])
tensor_aligned = tensor1.align_as(tensor2)

# Funciones que aceptan argumentos de dimensión permiten named dimensions:
suma_canales = tensor.sum('C')

# Si se intentan combinar dimensiones de distinto nombre se obtiene un error:
tensor_suma = tensor1 + tensor2

# Quitarle el nombre a un tensor:
tensor.rename(None)

### Broadcasting

In [ ]:
# Si uno de los operadores tiene dimensiones adicionales, el otro operador se repetirá en cada dimensión adicional:
a = torch.rand(2, 3, 4)
b = torch.rand(   3, 4)

(a*b).shape

In [ ]:
# Es necesario que la compatibilidad de dimensiones sea desde la derecha:
a = torch.rand(2, 3, 4)
b = torch.rand(2, 3)

try: (a*b).shape
except: print('No permitido.')

In [ ]:
# Si uno de los operadores tiene alguna dimensión de tamaño 1, se repetirá el operador en cada componente de la dimensión respectiva del otro operador:
a = torch.rand(5, 2, 3, 4, 1)
b = torch.rand(   2, 1, 4, 5)

(a*b + 1).shape

### Autograd

#### Cómputo de gradiente

In [ ]:
x1 = torch.tensor(10., requires_grad=True)
x2 = torch.tensor(100., requires_grad=True)

print(x1.grad, x2.grad)  # inicialmente los gradientes de x1 y x2 son None.

y = 3*x1 + 4*x2 + 5

# Por construcción, el tensor 'y' también irá guardando el grafo computacional para regresar hasta x1 y x2 en cadenas más largas:
print(y.requires_grad)  # si x1, x2 no guardaran grafo, y tampoco lo haría (basta que al menos uno lo haga).

# Se guarda la derivada (en cada parámetro) de quien hace el backward:
y.backward()  # se actualizarán los gradientes de x1 y x2 a dy/dx.

print(x1.grad, x2.grad)

In [ ]:
# Se acumulan (suman) los gradientes si no se reinician:

x = torch.tensor(10., requires_grad=True)

y1 = 3*x
y1.backward()  # se debería hacer x.grad.zero_()

y2 = 5*x
y2.backward()

print(x.grad) 

In [ ]:
# Como tensor 1D:
x = torch.tensor([10., 100], requires_grad=True)
y = torch.tensor([3., 4]) @ x

y.backward()
print(x.grad)

#### Regla de la cadena

In [ ]:
x1 = torch.tensor(10., requires_grad=True)
x2 = torch.tensor(100., requires_grad=True)
x3 = torch.tensor(1000., requires_grad=True)

y = 3*x1 + 4*x2 + 5
z = 6*y + 7*x3
# En los nodos hoja se guardará dz/dx. Para x1 y x2 se calculará como dz/dx = dz/dy * dy/dx.
z.backward()
# Gradientes propagados:
print(x1.grad, x2.grad, x3.grad, y.grad)  # no se actualiza el gradiente para nodos intermedios

In [ ]:
# En un backward solo se actualizan los gradientes para los nodos hoja (para ahorrar memoria). Es una decisión de diseño.
# Para actualizar los gradientes para los nodos intermedios, se debe indicar con retain_grad.

x1 = torch.tensor(10., requires_grad=True)
x2 = torch.tensor(100., requires_grad=True)
x3 = torch.tensor(1000., requires_grad=True)

y = 3*x1 + 4*x2 + 5
y.retain_grad()
z = 6*y + 7*x3
z.backward()
print(x1.grad, x2.grad, x3.grad, y.grad)

#### Operaciones in-place

In [ ]:
# Para operaciones in-place, se debe usar no_grad ya que pytorch no puede rastrear el cambio (puede necesitar valores que ya no están disponibles):

x = torch.tensor(5., requires_grad=True)

with torch.no_grad():  # sin esto se tendría un error.
    x += 2

y = x**2
y.backward()
print(x.grad)

# Si se hiciera una operación in-place despúes crear un nuevo tensor, no se podría propagar gradiente desde ese tensor:

x = torch.tensor(5., requires_grad=True)
y = x**2

with torch.no_grad():  # sin esto se tendría un error.
    x += 2

try: y.backward()
except: print('No permitido.')

#### Funcionamiento del optimizador

In [ ]:
from torch.optim import SGD

param = torch.tensor(100.0, requires_grad=True)

optimizer = SGD([param], lr=0.5)

output = 10 * param
optimizer.zero_grad() # reinicia los gradientes de los tensores registrados en el optimizador.
output.backward()

# Actualización:
optimizer.step()
print(param)

# Otra actualización (sin haber actualizado gradientes):
optimizer.step()
print(param)